# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**
    
    'сalls' — количество звонков,
    'minutes' — суммарная длительность звонков в минутах,
    'messages' — количество sms-сообщений,
    'mb_used' — израсходованный интернет-трафик в Мб,
    'is_ultra' — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Исследование проведем в соответствии с планом:**

    1) Откроем файл с данными и изучим общую информацию.
    2) Разделим исходные данные на обучающую, валидационную и тестовую выборки.
    3) Исследуем качество разных моделей, меняя гиперпараметры. Кратко напишием выводы исследования.
    4) Проверим качество модели на тестовой выборке.
    5) Проверим модели на вменяемость. 

## Откройте и изучите файл

**Откроем файл с данными и изучим общую информацию.**

### Импортируем необходимые для работы с проектом библиотеки.

In [1]:
# Импортируем библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Загрузим данные из файла в датафрейм.

In [2]:
# Создадим датафрейм
df=pd.read_csv('/datasets/users_behavior.csv')
# Ознакомимся с первыми 5 строками
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Изучим информацию о полученном датафрейме.

In [3]:
# общая информация
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


***

### Вывод по изучению общей информации о полученых данных:

<font color='green'> ***В ходе изучения полученных данных выявлено, что в полученном датафрейме имеется информация о 3214 пользователях, количестве звонков каждого пользователя и суммарной продолжительности этих звонков, количестве отправленных сообщений, объема исползованного трафика, а также информация об используемом тарифе. Предобработка данных не требуется.*** </font>

## Разбейте данные на выборки

**Разделим исходные данные на обучающую, валидационную и тестовую выборки.**

### Получим и проверим выборку для обучения модели.

In [4]:
# Создадим выборку
df_train,df_valid=train_test_split(df, test_size=0.4, random_state=12345)

# Проверим выборку
print("Обучающая выборка")
df_train.info()

Обучающая выборка
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 3027 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


### Получим и проверим валидационную и тестовую выборки для проверки и испытания обученной модели.

In [5]:
# Создадим выборки
df_valid, df_test=train_test_split(df_valid, test_size = 0.5, random_state=12345)

# Проверим валидационную выборку
print("Валидационная выборка")
df_valid.info()

# Проверим тестовую выборку
print()
print("Тестовая выборка")
df_test.info()

Валидационная выборка
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1386 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB

Тестовая выборка
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 160 to 2313
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


### Присвоим переменным признаки и целевой признак для валидационной и обучающих выборок.

In [6]:
# Присвоим признаки для обучающей выборки
features_train=df_train.drop(['is_ultra'], axis=1)
target_train=df_train['is_ultra']

# Присвоим признаки для валидационной выборки
features_valid=df_valid.drop(['is_ultra'], axis=1)
target_valid=df_valid['is_ultra']

***

### Вывод по разбитию данных на выборки:

<font color='green'> ***Для дальнейшей работы с моделью,  для ее обучения проверки и тестирования, нами были разделены исходные данные на обучающую (60%), валидационную (20%) и тестовую (20%) выборки.*** </font>

## Исследуйте модели

**Исследуем качество разных моделей, меняя гиперпараметры.**

### Исследуем модель решающего дерева.

In [7]:
# Проведем исследование модели с помощью цикла
best_result = 0
best_depth = 0

for depth in range(1, 10):
    model=DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid=model.predict(features_valid)
result=accuracy_score(target_valid, predictions_valid)

if result>=best_result:
    best_result=result
    best_depth=depth

'Лучшая глубина', best_depth, 'Лучшая модель', best_result

('Лучшая глубина', 9, 'Лучшая модель', 0.7822706065318819)

### Исследуем модель случайного леса.

In [8]:
# Проведем исследование модели с помощью цикла
best_result_forest=0
best_est=0

for est in range(1, 11):
    model_forest= RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    result=model_forest.score(features_valid, target_valid)
    
if result>=best_result_forest:
    best_result_forest=result
    best_est=est

"Количество деревьев в лесу", best_est, 'Лучшая модель', best_result_forest

('Количество деревьев в лесу', 10, 'Лучшая модель', 0.7853810264385692)

### Исследуем модель логистической регрессии.

In [9]:
# Проведем исследование модели с помощью цикла
solvers = ['lbfgs', 'liblinear', 'sag', 'saga']

best_result=0
best_solver=None

for solver in solvers:
    model_lr=LogisticRegression(random_state=12345, solver=solver, max_iter=10000)
    model_lr.fit(features_train, target_train)
    result=model_lr.score(features_valid, target_valid)
if result>=best_result:
    best_result=result
    best_solver=solver
print('Solver', best_solver, 'Лучшая модель', best_result)

Solver saga Лучшая модель 0.7076205287713841


***

### Вывод по исследования качеств разных моделей:

<font color='green'> **В ходе исследования качеств разных моделей с помощью подмены гиперпараметров было выявлено что, для выбранных моделей получились следующие результаты:**
* **Логистическая регрессия**: лучший гиперпараметр - `solver 'saga'` с результатом 0.7076205287713841
* **Лес**: оптимальное значение гиперпараметра `'Количество деревьев в лесу'`= 10, с результатом 0.7853810264385692
* **Дерево решений**: оптимальное значение гиперпараметра `'глубина'`= 9 c результатом 0.7822706065318819
    
**На основании полученных данных можно сделать вывод, что модель случайного леса показывает самую высокую точность и имеет смысл протестировать именно её.** </font>

## Проверьте модель на тестовой выборке

**Проверим качество выбранной модели на тестовой выборке.**

### Протестируем модель случайного леса. 

In [10]:
# Создадим тестовую выборку
test_features=df_test.drop(['is_ultra'], axis=1)
test_target=df_test['is_ultra']
# Протестируем модель
model_forest = RandomForestClassifier(n_estimators=10, random_state=12345)
model_forest.fit(features_train, target_train)
predictions_test=model_forest.predict(test_features)
accuracy_score(test_target, predictions_test)

0.7807153965785381

### Для улучшения результатов объеденим обучающую и валидационную выборки и протестируем модель повторно.

In [11]:
#Объеденим выборки
features_train = pd.concat([features_train, features_valid])
target_train = pd.concat([target_train, target_valid])
# Протестируем модель
model_forest = RandomForestClassifier(n_estimators=10, random_state=12345)
model_forest.fit(features_train, target_train)
predictions_test1=model_forest.predict(test_features)
accuracy_score(test_target, predictions_test1)

0.807153965785381

***

### Вывод о качестве моделей:

<font color='green'> ***После проверки значений `accuracy` на тестовой выборке для выбранной модели 'случайный лес', можно сделать вывод, что цель исследования достигнута. Доля правильных ответов модели превышает 80%.***

## (бонус) Проверьте модели на адекватность

In [12]:
df['is_ultra'].value_counts()

df['is_ultra'].value_counts(normalize=True)

df_test['is_ultra'].value_counts()

df_test['is_ultra'].value_counts(normalize=True)

0    0.684292
1    0.315708
Name: is_ultra, dtype: float64

***

# *Вывод*:

<font color='green'> ***По итогу исследования для задачи классификации при построении системы, способной проанализировать поведение клиентов и предложить пользователям оператора мобильной связи «Мегалайн» новый тариф, была выбрана модель Случайного леса, которая показала лучший результат на тестовой выборке. Таким образом, модель предложит перейти на тариф: «Смарт» или «Ультра» абонентам, которым это действительно нужно, c вероятностью 80%.***

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
